# 라이브러리

In [2]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if cuda.is_available() else 'cpu'
path = "/content/drive/MyDrive/21_winter/CIFAR-100/cifar-100-python"
batch_size = 32

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 불러오기

In [25]:
train_data.keys()

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])

In [43]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

meta_data = unpickle(path + "/meta")
train_data = unpickle(path + "/train")
test_data = unpickle(path + "/test")

train_data_data = train_data[b'data']
train_data_label_1 = train_data[b'coarse_labels']
train_data_label_2 = train_data[b'fine_labels']

test_data_data = test_data[b'data']
test_data_label_1 = test_data[b'coarse_labels']
test_data_label_2 = test_data[b'fine_labels']


# 데이터 준비

In [48]:
class GH_Dataset_train(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((50000,3,32,32))
        self.y_data = torch.tensor(Y).resize_(50000,1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

class GH_Dataset_test(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((10000,3,32,32))
        self.y_data = torch.tensor(Y).resize_(10000,1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = GH_Dataset_train(train_data_data,train_data_label_1)
train_loader = data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = GH_Dataset_test(test_data_data,test_data_label_1)
test_loader = data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# 모델

In [49]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.squeeze(target))
        loss.backward()
        optimizer.step()
        if batch_idx %100000 == 0:
            print('==================\nTrain Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, torch.squeeze(target)).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print(f'Test set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


# 모델 학습

In [50]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Tesing time: {m:.0f}m {s:.0f}s')
        
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Train Epoch : 1 | Batch Status : 0/50000 (0%) | Loss : 12.757238
Training time: 2m 15s
Test set: Average loss : 0.0406, Accuracy : 5867/10000(59%)
Tesing time: 2m 22s
Train Epoch : 2 | Batch Status : 0/50000 (0%) | Loss : 1.214159
Training time: 2m 14s
Test set: Average loss : 0.0356, Accuracy : 6373/10000(64%)
Tesing time: 2m 21s
Train Epoch : 3 | Batch Status : 0/50000 (0%) | Loss : 0.841067
Training time: 2m 14s
Test set: Average loss : 0.0333, Accuracy : 6645/10000(66%)
Tesing time: 2m 21s
Train Epoch : 4 | Batch Status : 0/50000 (0%) | Loss : 0.949539
Training time: 2m 14s
Test set: Average loss : 0.0320, Accuracy : 6856/10000(69%)
Tesing time: 2m 21s
Train Epoch : 5 | Batch Status : 0/50000 (0%) | Loss : 0.427996
Training time: 2m 14s
Test set: Average loss : 0.0374, Accuracy : 6559/10000(66%)
Tesing time: 2m 21s
Train Epoch : 6 | Batch Status : 0/50000 (0%) | Loss : 0.452651
Training time: 2m 14s
Test set: Average loss : 0.0350, Accuracy : 6742/10000(67%)
Tesing time: 2m 21s
Tra